<a href="https://colab.research.google.com/github/ss-ghule/kaggle_predicting_molecular_properties/blob/master/c4_w2_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Aim: To create a traing and test dataframes for downstream precesses

# Load & Extract Data

In [0]:
# The code was removed by Watson Studio for sharing.

In [1]:
#load_data()
!ls

kaggle.json  sample_data


In [2]:
!pwd

/content


In [3]:
%env KAGGLE_CONFIG_DIR=/content
!echo $KAGGLE_CONFIG_DIR

env: KAGGLE_CONFIG_DIR=/content
/content


In [4]:
!kaggle competitions download -c champs-scalar-coupling

 92% 41.0M/44.5M [00:01<00:00, 28.4MB/s]
100% 44.5M/44.5M [00:01<00:00, 40.0MB/s]
 67% 9.00M/13.4M [00:00<00:00, 15.9MB/s]
100% 13.4M/13.4M [00:00<00:00, 22.8MB/s]
 94% 59.0M/62.9M [00:01<00:00, 27.3MB/s]
100% 62.9M/62.9M [00:01<00:00, 56.5MB/s]
 84% 41.0M/48.7M [00:01<00:00, 22.4MB/s]
100% 48.7M/48.7M [00:01<00:00, 38.7MB/s]
 99% 86.0M/86.6M [00:01<00:00, 36.2MB/s]
100% 86.6M/86.6M [00:01<00:00, 49.9MB/s]
 94% 5.00M/5.31M [00:00<00:00, 13.6MB/s]
100% 5.31M/5.31M [00:00<00:00, 13.4MB/s]
  0% 0.00/497k [00:00<?, ?B/s]
100% 497k/497k [00:00<00:00, 165MB/s]
 54% 5.00M/9.24M [00:00<00:00, 15.5MB/s]
100% 9.24M/9.24M [00:00<00:00, 26.6MB/s]
 82% 41.0M/49.8M [00:01<00:00, 20.6MB/s]
100% 49.8M/49.8M [00:01<00:00, 41.2MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 133MB/s]


In [5]:
!ls

dipole_moments.csv.zip		    sample_submission.csv.zip
kaggle.json			    scalar_coupling_contributions.csv.zip
magnetic_shielding_tensors.csv.zip  structures.csv.zip
mulliken_charges.csv.zip	    structures.zip
potential_energy.csv.zip	    test.csv.zip
sample_data			    train.csv.zip


# Basic Transfomations

## Training data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
train=pd.read_csv('train.csv.zip')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [8]:
structures=pd.read_csv('structures.csv.zip')
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [9]:
charges=pd.read_csv('mulliken_charges.csv.zip')
charges.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [10]:
structures_0=structures.rename(columns={'atom_index':'atom_index_0','atom':'atom_0','x':'x_0','y':'y_0','z':'z_0'})
structures_0.head()

,molecule_name,atom_index_0,atom_0,x_0,y_0,z_0
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [11]:
train=pd.merge(train,structures_0,on=['molecule_name','atom_index_0'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277


In [12]:
structures_1=structures.rename(columns={'atom_index':'atom_index_1','atom':'atom_1','x':'x_1','y':'y_1','z':'z_1'})
structures_1.head()

,molecule_name,atom_index_1,atom_1,x_1,y_1,z_1
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [13]:
train=pd.merge(train,structures_1,on=['molecule_name','atom_index_1'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001


In [14]:
train.shape

(4658147, 14)

In [15]:
charges.head()

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [16]:
charges_0=charges.rename(columns={'atom_index':'atom_index_0','mulliken_charge':'mulliken_charge_0'})
charges_0.head()

,molecule_name,atom_index_0,mulliken_charge_0
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [17]:
train=pd.merge(train,charges_0,on=['molecule_name','atom_index_0'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922


In [18]:
charges_1=charges.rename(columns={'atom_index':'atom_index_1','mulliken_charge':'mulliken_charge_1'})
charges_1.head()

,molecule_name,atom_index_1,mulliken_charge_1
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [19]:
train=pd.merge(train,charges_1,on=['molecule_name','atom_index_1'],how='left')
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689


In [20]:
train.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type',
       'scalar_coupling_constant', 'atom_0', 'x_0', 'y_0', 'z_0', 'atom_1',
       'x_1', 'y_1', 'z_1', 'mulliken_charge_0', 'mulliken_charge_1'],
      dtype='object')

In [0]:
train=train[['id', 'molecule_name', 'atom_index_0', 'atom_index_1',
              'atom_0', 'x_0', 'y_0', 'z_0', 
              'atom_1', 'x_1', 'y_1', 'z_1',
              'mulliken_charge_0', 'mulliken_charge_1',
              'type','scalar_coupling_constant']]

In [22]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,H,0.002150,-0.006031,0.001976,C,-0.012698,1.085804,0.008001,0.133921,-0.535689,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,H,0.002150,-0.006031,0.001976,H,1.011731,1.463751,0.000277,0.133921,0.133922,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,H,0.002150,-0.006031,0.001976,H,-0.540815,1.447527,-0.876644,0.133921,0.133923,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,H,0.002150,-0.006031,0.001976,H,-0.523814,1.437933,0.906397,0.133921,0.133923,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,H,1.011731,1.463751,0.000277,C,-0.012698,1.085804,0.008001,0.133922,-0.535689,1JHC,84.8074


In [23]:
train.shape

(4658147, 16)

## Test data

In [24]:
test=pd.read_csv('test.csv.zip')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [25]:
test=pd.merge(test,structures_0,on=['molecule_name','atom_index_0'],how='left')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0


In [26]:
test=pd.merge(test,structures_1,on=['molecule_name','atom_index_1'],how='left')
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0


In [0]:
test['mulliken_charge_0']=np.nan
test['mulliken_charge_1']=np.nan

In [28]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1
0,4658147,dsgdb9nsd_000004,2,0,2JHC,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN
1,4658148,dsgdb9nsd_000004,2,1,1JHC,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN
2,4658149,dsgdb9nsd_000004,2,3,3JHH,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN
3,4658150,dsgdb9nsd_000004,3,0,1JHC,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN
4,4658151,dsgdb9nsd_000004,3,1,2JHC,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN


In [29]:
test.shape

(2505542, 15)

In [30]:
test.columns

Index(['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'type', 'atom_0',
       'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1'],
      dtype='object')

In [0]:
test=test[['id', 'molecule_name', 'atom_index_0', 'atom_index_1', 'atom_0',
       'x_0', 'y_0', 'z_0', 'atom_1', 'x_1', 'y_1', 'z_1', 'mulliken_charge_0',
       'mulliken_charge_1','type']]

In [32]:
test.head()

,id,molecule_name,atom_index_0,atom_index_1,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,mulliken_charge_0,mulliken_charge_1,type
0,4658147,dsgdb9nsd_000004,2,0,H,-1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,2JHC
1,4658148,dsgdb9nsd_000004,2,1,H,-1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,1JHC
2,4658149,dsgdb9nsd_000004,2,3,H,-1.661639,0.0,1.0,H,1.661639,0.0,1.0,NaN,NaN,3JHH
3,4658150,dsgdb9nsd_000004,3,0,H,1.661639,0.0,1.0,C,0.599539,0.0,1.0,NaN,NaN,1JHC
4,4658151,dsgdb9nsd_000004,3,1,H,1.661639,0.0,1.0,C,-0.599539,0.0,1.0,NaN,NaN,2JHC


In [33]:
test.shape

(2505542, 15)

In [34]:
train.shape

(4658147, 16)

# Save Train and Test Data to disk

In [0]:
train.to_csv('train_etl.csv',index=False)
test.to_csv('test_etl.csv',index=False)

In [36]:
!ls

dipole_moments.csv.zip		    scalar_coupling_contributions.csv.zip
kaggle.json			    structures.csv.zip
magnetic_shielding_tensors.csv.zip  structures.zip
mulliken_charges.csv.zip	    test.csv.zip
potential_energy.csv.zip	    test_etl.csv
sample_data			    train.csv.zip
sample_submission.csv.zip	    train_etl.csv


# Upload train and test data to the cloud object store

In [0]:
# The code was removed by Watson Studio for sharing.

In [38]:
'''
from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded')
'''

"\nfrom ibm_botocore.client import Config\nimport ibm_boto3\ndef upload_file_cos(credentials,local_file_name,key):  \n    cos = ibm_boto3.client(service_name='s3',\n    ibm_api_key_id=credentials['IBM_API_KEY_ID'],\n    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],\n    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],\n    config=Config(signature_version='oauth'),\n    endpoint_url=credentials['ENDPOINT'])\n    try:\n        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)\n    except Exception as e:\n        print(Exception, e)\n    else:\n        print('File Uploaded')\n"

In [0]:
#upload_file_cos(credentials_2,'train_etl.csv','train_etl.csv')

In [0]:
#upload_file_cos(credentials_2,'test_etl.csv','test_etl.csv')